# TP2- Introducción al SGBD MongoDB y Conexión a la API Rest de Twitter

En este trabajo práctico queremos descubrir las características del SGBD no relacional MongoDB. En este sentido, vamos a desarrollar un script que se conecta a la API Rest de Twitter para: 1) descargar datos al formato JSON, 2) almacenarlos en una base Mongo y 3) realizar algunas consultas sobre los datos.

## 1. Conectarse a la API Rest de Twitter

- ¿Cómo crear una aplicación utilizando los datos de Twitter?: https://developer.twitter.com/en/docs/basics/getting-started
- Crear una aplicación utilizando los datos de Twitter: https://apps.twitter.com/
- ¿Qué deberían saber todos los desarrolladores antes de jugar con la API de twitter?: https://developer.twitter.com/en/docs/basics/things-every-developer-should-know

In [154]:
import tweepy

consumer_key="s5CHuCpX18z66qXBXM6zSnufK"
consumer_secret="lCGAR8m5ybxXMV4Dq4h4Jn1qXfaZhWj6PhQiMOe1yVd1d0RE6e"
access_token="907630424758898688-F0KnxCXc7jOLHBA1FRvVE6Ze0YSg7or"
access_secret="uZKmionfgxfD5PTfPQMph8l09fF5Mz8ikaF2QzrYQkIpP"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## 2. Consultar la API de Twitter

- Utilizando una librería Python que permite enviar consultas HTTP a la API de Twitter, conectarse a un 'endpoint' de la API para leer datos. Ver la lista de los endpoints: https://developer.twitter.com/en/docs/tweets/post-and-engage/overview

In [155]:
# tweets=api.search(q='valdivia', count=500) asi se busca, pero mas abajo se realizara

## 3. Conectarse al SGBD Mongo

- Conectarse a Mongo desde Python utilizando la librería adecuada

In [156]:

from pymongo import MongoClient
mc= MongoClient('localhost', 27017)
db=mc.test
db_tweets = db.tweets



## 4. Escribir datos de Twitter en una Base de Datos Mongo

Tener una colección Mongo almacendando Documentos con esta estructura JSON:
{
 fecha: String
 tweet: String
 username: String
 location: String
}

5. Almacenar 500 tweets con los datos pedidos

In [157]:
import time
since_id=0
db_tweets.delete_many({})
for i in range(5):
    print("recolectando tweets #{0} intento".format(i))
    print('Buscando tweets desde la id: {0}'.format(since_id))
    tweets=api.search(q='valdivia',since_id=since_id,count=100)
    tweets_id=[]
    print("{0} tweets recolectados".format(len(tweets)))
    print("Almacenando tweets en mongoDB..")
    for tweet in tweets:
        tweets_id.append(tweet.id)
        new_tweet={
                "id":tweet.id,
                "tweet_text":tweet.text,
                "date":tweet.created_at,
                "user_name":tweet.user.screen_name,
                "location": tweet.user.location
        }
        db_tweets.insert_one(new_tweet)
        print("tweet id [{0}] almacenado".format(tweet.id))
    since_id=max(tweets_id)
    #print(tweets_id)
    print("Esperando 2 minutos para volver a consultar...")
    time.sleep(2*60)

    

recolectando tweets #0 intento
Buscando tweets desde la id: 0
100 tweets recolectados
Almacenando tweets en mongoDB..
tweet id [985294777108324352] almacenado
tweet id [985294730085961739] almacenado
tweet id [985294595323060225] almacenado
tweet id [985294546794893312] almacenado
tweet id [985294499386716160] almacenado
tweet id [985294338321207298] almacenado
tweet id [985294309246361601] almacenado
tweet id [985294294553710596] almacenado
tweet id [985294189603811329] almacenado
tweet id [985294186890002432] almacenado
tweet id [985294175661973509] almacenado
tweet id [985294171597672448] almacenado
tweet id [985294113225441281] almacenado
tweet id [985294074159693824] almacenado
tweet id [985294017385713665] almacenado
tweet id [985294005956116480] almacenado
tweet id [985293786132697090] almacenado
tweet id [985293778234650624] almacenado
tweet id [985293650103013377] almacenado
tweet id [985293483434012672] almacenado
tweet id [985293413452079104] almacenado
tweet id [98529333988

Parte I: Desde su notebook Python (o desde la CLI de Mongo en un terminal, o la GUI Mongo Compass), realizar las consultas siguientes:

 Mostrar los documentos de su colección mostrando solamente el campo "location". 

In [158]:
result = db.tweets.find({})
for tweet in result:
    print(tweet['location'])

Valdivia
Chile, sur
Valdivia - Chile
Nicaragua
www.deportereyydereyes.cl
Lima, Peru



Valdivia- Chile



Buenos Aires, Argentina

Sleepy Hollow y Calalini
SP, Brazil
Lima, Peru

Calama, Chile
Valdivia-Chile
Lissabon, Portugal

La Serena, Coquimbo
Cuiabá-MT



Puerto Montt - Chile





Puerto Montt, Chile
Puerto Montt, Chile
lucanoland
Santiago de Chile
US, Chile and every where
Brasília, Brasil
Valdivia, Chile

Yungay 499, Valdivia

Chile

Santiago Chile

Puerto Montt
Chile
Santiago, Chile
Medellín, Colombia

Valdivia-Chile

Chile
Chile



São Mateus, SP
Chile
Chile, Los Rios
Chile

Valdivia, Chile
Brasília, Brasil

Chile
São Paulo

Chile

Temuco, Chile

Valparaiso,Chile
Valdivia, Chile





Puerto Montt


Chile

-33.460545,-70.61319

Chile
Valdivia
Chile
Valdivia, Chile
La Reina, Chile
Valdivia, Chile

Chile


Valdivia, Chile
La Serena, Coquimbo


Cuiabá-MT
Venezuela♥/Por ahora en Chile 

Perú
La Serena


Mostrar los tweets que contienen la palabra "UACh".

In [202]:
result = db.tweets.find({'tweet_text':{'$regex':'UACh',"$options":"$i"}})
print(result.count())
for tweet in result:
    print(tweet['tweet_text'])

0


El i es para no distinguir entre mayúsculas y minúsculas

Contar el número de tweets por username.

In [160]:
result = db.tweets.aggregate([{"$group":{"_id":"$user_name","total":{"$sum":1}}}])
for tweet in result:
    print(tweet)

{'_id': 'Daniel_San_39', 'total': 1}
{'_id': 'perez_indio', 'total': 1}
{'_id': 'gidion1208', 'total': 1}
{'_id': 'Rockesvida', 'total': 1}
{'_id': 'mess70', 'total': 1}
{'_id': 'FUbiergo', 'total': 1}
{'_id': 'valdivia_herman', 'total': 1}
{'_id': 'leandro_paredes', 'total': 1}
{'_id': 'marfe230596', 'total': 1}
{'_id': 'jctedeschi', 'total': 1}
{'_id': 'adeurresti', 'total': 1}
{'_id': 'leigchile', 'total': 1}
{'_id': 'IsaSerey', 'total': 1}
{'_id': 'luiz_slovak', 'total': 1}
{'_id': 'RumboaPrimeraCL', 'total': 1}
{'_id': 'CoolTV_Valdivia', 'total': 1}
{'_id': 'enelcamarin', 'total': 3}
{'_id': 'hernaneszz', 'total': 1}
{'_id': 'ortegabalboa', 'total': 1}
{'_id': 'karlotaroa', 'total': 1}
{'_id': 'legispe', 'total': 1}
{'_id': '14coto', 'total': 1}
{'_id': 'SciForProgress', 'total': 1}
{'_id': 'soyvaldiviacl', 'total': 1}
{'_id': 'crisvsanchezv', 'total': 1}
{'_id': 'primerabdechile', 'total': 8}
{'_id': 'gustastockler', 'total': 1}
{'_id': 'paakyy', 'total': 1}
{'_id': 'Valdivia_10'

Parte II: El SGBD Mongo permite realizar consultas geospaciales (https://docs.mongodb.com/manual/geospatial-queries/)

1) En una sola consulta, actualizar los documentos tweets que hablan de "valdiviacl", agregando un campo "city" con el valor siguiente:
{
 type: "Point",
 coordinates: [-73.245209,-39.819588]
}

In [161]:
db.tweets.update_many( {'tweet_text':{'$regex':'valdiviacl', "$options":"$i"}}, {'$set': {'city':{'type':'Point','coordinates':[-73.245209,-39.819588]}}})


2) Recopilar algunos tweets hablando de "Puerto Montt" y agregar un campo "city" con las coordinades de Puerto Montt

In [162]:
db.tweets.update_many( {'tweet_text':{'$regex':'Puerto Montt', "$options":"$i"}}, {'$set': {'city':{'type':'Point','coordinates':[-41.462561,-72.9988162]}}})


3) Utilizar un operator de Mongo para recopilar el documento lo más cercano del punto: -73.131920, -40.574505

In [211]:
#db.tweets.createIndex( { 'loc' : "2dsphere" } )
t=db.tweets.find( {
   'loc': {
     '$near': {
       '$geometry': {
          'type': "Point" ,
          'coordinates': [-73.131920, -40.574505 ]
       },
       '$maxDistance': 5000,
       '$minDistance': 1000
     }
   }
})
print(t)